In [16]:
import csv
import glob
from iso639 import Lang
import pycountry
from langcodes import *
import os
import pandas as pd

def addToDict(word, reps,dictionary):
    if word in dictionary:
        dictionary[word]+=reps
    else:
        dictionary[word]=reps


csvsToAnalyze=glob.glob("./../../../data/non_filtered/corpora/pud/*.csv")
for csvfile in csvsToAnalyze:
    with open(csvfile, newline='',encoding="utf-8") as csvfileOpened:
        reader = csv.reader(csvfileOpened, delimiter=',', quotechar='"')
        next(reader)
        names=csvfile.split("\\")[1].split("_")[0]
        isoLan=names
        language=pycountry.languages.get(alpha_3=Lang(isoLan).pt3)
        isoLanMod=language.name
        isoLanIdentifier=names
        isoLan=Lang(isoLanIdentifier).pt1
        characterFreq={}
        totLetters=0
        if "pinyin" in csvfile:
            characterFreq={}
            for row in reader:
                for letter in row[4]:
                    totLetters+=int(row[2])
                    addToDict(letter,int(row[2]),characterFreq)
        if "romaji" in csvfile:
            for row in reader:
                for letter in row[4]:
                    totLetters+=int(row[2])
                    addToDict(letter,int(row[2]),characterFreq)
        else:
            for row in reader:
                for letter in row[0]:
                    totLetters+=int(row[2])
                    addToDict(letter,int(row[2]),characterFreq)
        language=pycountry.languages.get(alpha_3=Lang(isoLan).pt3)
        toAdd=""
        if "strokes" in csvfile:
            toAdd+="-strokes"
        if "pinyin" in csvfile:
            toAdd+="-pinyin"
        if "romaji" in csvfile:
            toAdd+= "-romaji"

        with open("./../../../data/non_filtered/alphabets/pud/"+Lang(isoLan).pt3+toAdd+"-character.csv", 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["character", "frequencyTot", "frequencyRet"])
            for w in sorted(characterFreq, key=characterFreq.get, reverse=True):
                    writer.writerow([w, characterFreq[w], characterFreq[w]/totLetters])


{'d': 3401, 'e': 1971, 'z': 2979, 'à': 3305, 'i': 5149, 'l': 1834, 'y': 2799, 'ī': 1754, 's': 2554, 'h': 5886, 'ì': 2989, 'é': 1742, 'n': 9752, 'á': 1666, 'g': 5736, 'è': 1946, 't': 1202, 'ā': 2201, 'w': 849, 'ú': 629, 'ō': 1014, 'ǒ': 765, 'u': 3755, 'b': 1337, 'r': 877, 'm': 972, 'ě': 778, 'ù': 1364, 'j': 1944, 'ǔ': 529, 'ǎ': 1560, 'c': 1103, 'ó': 701, 'q': 912, 'í': 1388, 'ò': 940, 'ǐ': 1062, 'o': 1247, 'k': 596, 'x': 1455, 'ē': 803, 'ū': 520, 'f': 789, 'p': 324, 'ǜ': 30, 'ǚ': 41, 'a': 5, 'ü': 10, 'ǘ': 2}
